In [31]:
!pip install beautifulsoup4 

In [32]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
!pip install contractions
import contractions

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/

In [33]:
import os
import re
import string
import emoji
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [34]:
train_data = pd.read_csv("../input/goemotions/data/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
valid_data = pd.read_csv("../input/goemotions/data/dev.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [35]:
train_data.head()

,Text,Class,ID
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


In [36]:
# train_data.drop("ID", axis=1, inplace=True)

In [37]:
train_data['Class List'] = train_data['Class'].apply(lambda x: x.split(','))
train_data['Class Length'] = train_data['Class List'].apply(lambda x: len(x))

valid_data['Class List'] = valid_data['Class'].apply(lambda x: x.split(','))
valid_data['Class Length'] = valid_data['Class List'].apply(lambda x: len(x))

In [38]:
with open('../input/goemotions/data/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

In [39]:
emotion_file = open("../input/goemotions/data/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [40]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [41]:
train_data['Emotions'] = train_data['Class List'].apply(idx2class)
valid_data['Emotions'] = valid_data['Class List'].apply(idx2class)

In [42]:
def EmotionMapping(emotion_list):
    map_list = []
    
    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['disgust']:
            map_list.append('disgust')
        if i in ekman_mapping['fear']:
            map_list.append('fear')
        if i in ekman_mapping['joy']:
            map_list.append('joy')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')
            
    return map_list

In [43]:
train_data['Mapped Emotions'] = train_data['Emotions'].apply(EmotionMapping)
valid_data['Mapped Emotions'] = valid_data['Emotions'].apply(EmotionMapping)

In [44]:
train_data['anger'] = np.zeros((len(train_data),1))
train_data['disgust'] = np.zeros((len(train_data),1))
train_data['fear'] = np.zeros((len(train_data),1))
train_data['joy'] = np.zeros((len(train_data),1))
train_data['sadness'] = np.zeros((len(train_data),1))
train_data['surprise'] = np.zeros((len(train_data),1))
train_data['neutral'] = np.zeros((len(train_data),1))

valid_data['anger'] = np.zeros((len(valid_data),1))
valid_data['disgust'] = np.zeros((len(valid_data),1))
valid_data['fear'] = np.zeros((len(valid_data),1))
valid_data['joy'] = np.zeros((len(valid_data),1))
valid_data['sadness'] = np.zeros((len(valid_data),1))
valid_data['surprise'] = np.zeros((len(valid_data),1))
valid_data['neutral'] = np.zeros((len(valid_data),1))

In [45]:
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    train_data[i] = train_data['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    valid_data[i] = valid_data['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [46]:
# train_data.drop(train_data[train_data['neutral'] == 1].index, inplace=True)
# valid_data.drop(valid_data[valid_data['neutral'] == 1].index, inplace=True)

In [47]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0,0


In [48]:
train_data.drop(["Class", "ID", "Class List", "Class Length", "Emotions", "Mapped Emotions"], axis=1, inplace=True)
valid_data.drop(["Class", "ID", "Class List", "Class Length", "Emotions", "Mapped Emotions"], axis=1, inplace=True)

In [49]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [50]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
#     text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub('[^A-z]', ' ', sentence)
    negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                        'even though', 'yet']
    stop_words = [z for z in stop_words if z not in negative]
    preprocessed_tokens = [lemmatizer.lemmatize(contractions.fix(temp.lower())) for temp in sentence.split() if temp not in stop_words] #lemmatization
    return ' '.join([x for x in preprocessed_tokens]).strip()

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    text = preprocess(text)
    return text

In [51]:
train_data['Text'] = train_data['Text'].apply(text_preprocessing_pipeline)

In [52]:
valid_data["Text"] = valid_data["Text"].apply(text_preprocessing_pipeline)

In [53]:
pd.Series(train_data["Text"]).str.split().str.len().describe(percentiles=[0.05, 0.97])

count    43410.000000
mean         6.595715
std          3.603889
min          0.000000
5%           2.000000
50%          6.000000
97%         14.000000
max         26.000000
Name: Text, dtype: float64

In [54]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(
    num_words=None,
    lower=True,
    split=" ",
    oov_token=""
)

In [55]:
train_text_abs = train_data["Text"].values
tokenizer.fit_on_texts(train_text_abs)
train_raw_sequences = tokenizer.texts_to_sequences(train_text_abs)

valid_text_abs = valid_data["Text"].values
valid_raw_sequences = tokenizer.texts_to_sequences(valid_text_abs)

In [56]:
max_seq_len = 25
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_raw_sequences,
                                                          maxlen=max_seq_len,
                                                          padding='post',
                                                          truncating='post')

valid_sequences = tf.keras.preprocessing.sequence.pad_sequences(valid_raw_sequences,
                                                          maxlen=max_seq_len,
                                                          padding='post',
                                                          truncating='post')

In [57]:
train_text, train_labels = train_data["Text"], train_data.drop("Text", axis=1).values
train_features = np.array([item for item in [seq for seq in train_sequences]])

valid_text, valid_labels = valid_data["Text"], valid_data.drop("Text", axis=1).values
valid_features = np.array([item for item in [seq for seq in valid_sequences]])

In [58]:
train_features.shape

(43410, 25)

In [59]:
import tensorflow.keras.layers as layers

n_vocab = len(tokenizer.index_word) + 1

word_inputs = layers.Input(shape=(max_seq_len), dtype="int32")
embedding_out = layers.Embedding(input_dim=n_vocab, output_dim=128)(word_inputs)
bls_1 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embedding_out)
bls_2 = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(bls_1)
hidden_layer = layers.Dense(256, activation="relu")(bls_2)
dropout_layer = layers.Dropout(0.7)(hidden_layer)
out = layers.Dense(7, activation="sigmoid")(dropout_layer)

model = tf.keras.models.Model(inputs=word_inputs, outputs=out)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25)]              0         
                                                                 
 embedding (Embedding)       (None, 25, 128)           2944896   
                                                                 
 bidirectional (Bidirectiona  (None, 25, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [60]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer="adam",
              metrics=["accuracy", tf.keras.metrics.Precision(name='precision')])

In [61]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [62]:
history = model.fit(train_features, train_labels,
                    epochs=100,
                    validation_data=(valid_features, valid_labels),
                    batch_size=3000,
                    callbacks=[es])

Epoch 1/100
15/15 [==============================] - 17s 397ms/step - loss: 0.5417 - accuracy: 0.2960 - precision: 0.2835 - val_loss: 0.3727 - val_accuracy: 0.3953 - val_precision: 0.0000e+00
Epoch 2/100
15/15 [==============================] - 4s 244ms/step - loss: 0.3938 - accuracy: 0.3424 - precision: 0.3789 - val_loss: 0.3603 - val_accuracy: 0.3953 - val_precision: 0.0000e+00
Epoch 3/100
15/15 [==============================] - 4s 234ms/step - loss: 0.3785 - accuracy: 0.3570 - precision: 0.3889 - val_loss: 0.3570 - val_accuracy: 0.3953 - val_precision: 0.0000e+00
Epoch 4/100
15/15 [==============================] - 3s 181ms/step - loss: 0.3730 - accuracy: 0.3606 - precision: 0.3887 - val_loss: 0.3560 - val_accuracy: 0.3953 - val_precision: 0.0000e+00
Epoch 5/100
15/15 [==============================] - 3s 223ms/step - loss: 0.3687 - accuracy: 0.3670 - precision: 0.4134 - val_loss: 0.3544 - val_accuracy: 0.3953 - val_precision: 0.0000e+00
Epoch 6/100
15/15 [=========================

In [63]:
y_pred = model.predict(valid_features)

170/170 [==============================] - 2s 5ms/step


In [64]:
model.evaluate(valid_features, valid_labels)

170/170 [==============================] - 1s 6ms/step - loss: 0.3021 - accuracy: 0.5562 - precision: 0.6808


[0.3021496534347534, 0.5562108159065247, 0.6807610988616943]

In [66]:
y_pred

array([[0.06488924, 0.00891702, 0.01069465, ..., 0.04790815, 0.16547757,
        0.14655852],
       [0.15547006, 0.00377242, 0.00471019, ..., 0.02179714, 0.18278523,
        0.6565219 ],
       [0.31107756, 0.11967677, 0.09692574, ..., 0.46150517, 0.09498016,
        0.03169348],
       ...,
       [0.12015469, 0.00369633, 0.00445752, ..., 0.02128216, 0.17955227,
        0.676832  ],
       [0.07439656, 0.00217835, 0.00242692, ..., 0.0126291 , 0.14106259,
        0.7471978 ],
       [0.043118  , 0.0030398 , 0.00358339, ..., 0.01835004, 0.12387992,
        0.32739133]], dtype=float32)

In [67]:
def prob(x):
    if x>0.4:
        return 1
    return 0

pred_labels = [list(map(prob,i)) for i in y_pred]

In [68]:
from sklearn.metrics import classification_report
target_names = train_data.columns[1:]
print(classification_report(valid_labels, pred_labels, target_names=target_names))

              precision    recall  f1-score   support

       anger       0.40      0.12      0.19       717
     disgust       0.00      0.00      0.00        97
        fear       0.00      0.00      0.00       105
         joy       0.74      0.76      0.75      2219
     sadness       0.50      0.28      0.36       390
    surprise       0.00      0.00      0.00       624
     neutral       0.55      0.60      0.57      1766

   micro avg       0.63      0.50      0.56      5918
   macro avg       0.31      0.25      0.27      5918
weighted avg       0.52      0.50      0.50      5918
 samples avg       0.53      0.51      0.52      5918

